In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/home/saloni/Documents/ML/Mortality/train.csv')
labels=pd.read_csv('/home/saloni/Documents/ML/Mortality/labels_train.csv')


In [4]:
dailydata=df.copy()
result=pd.concat([dailydata,labels],axis=1)

In [5]:
result.head(1)

,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,DiasABP,...,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,In-hospital_death
0,77.0,31.0,46.0,54,2.973333,10.5,0.7,154.0,0.75,58.795833,...,97.25,116.891892,37.357143,2.1,0.14,171.052632,10.3,80.060976,7.387273,0


In [6]:
list_of_variables = ['Age', 'Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol', 'Creatinine',
                             'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'Height', 'HR', 'ICUType', 'K',
                             'Lactate', 'Mg', 'MAP', 'MechVent', 'Na', 'NIDiasABP', 'NIMAP', 'NISysABP', 'PaCO2', 'PaO2',
                             'pH', 'Platelets', 'RecordID', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI','TroponinT','Urine',
                             'WBC', 'Weight']

In [7]:
datawithoutoutliers=result.copy()
    

In [8]:
for i in list_of_variables:
    datawithoutoutliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())<=(4*datawithoutoutliers[i].std())]
    outliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())>=(4*datawithoutoutliers[i].std())]
    
print(datawithoutoutliers.shape)

(3169, 43)


In [9]:
labels=datawithoutoutliers.pop("In-hospital_death")
df=datawithoutoutliers.copy()

In [10]:
labels.shape


(3169,)

In [11]:
df.shape

(3169, 42)

In [12]:
from sklearn.preprocessing import MinMaxScaler
#normalization
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)


In [13]:
from imblearn.over_sampling import SMOTE

df_res,labels_res=SMOTE(random_state=9).fit_sample(df,labels) #balancing the data
df_res=pd.DataFrame(df_res)
labels_res=pd.DataFrame(labels_res)

In [14]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
onehot_encoded = enc.fit_transform(labels_res)

labels_res=onehot_encoded

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_res,labels_res,test_size=0.33,random_state=42)


In [16]:
sess=tf.InteractiveSession()

In [17]:
n_hidden_1=80
n_hidden_2=80
n_hidden_3=80
n_hidden_4=80
n_hidden_5=80


x=tf.placeholder(tf.float32,shape=[None,42])
y_=tf.placeholder(tf.float32,shape=[None,2])
w =tf.Variable(tf.zeros([42,2]))
b=tf.Variable(tf.zeros([2]))

def multilayer_perceptron(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.sigmoid(layer_3)
    
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.sigmoid(layer_4)
    
    layer_5=tf.add(tf.matmul(layer_4,weights['h5']),biases['b5'])
    layer_5=tf.nn.relu(layer_5)
    
    
    out_layer=tf.add(tf.matmul(layer_5,weights['out']),biases['out'])
    return out_layer

weights={
       
    'h1':tf.Variable(tf.truncated_normal([42,n_hidden_1])),
    'h2':tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'h5':tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_5])),
    'out':tf.Variable(tf.truncated_normal([n_hidden_5,2]))
}

biases={
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
    'b5':tf.Variable(tf.truncated_normal([n_hidden_5])),
    'out':tf.Variable(tf.truncated_normal([2]))
}
    
y=multilayer_perceptron(x,weights,biases)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))  
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
sess.run(tf.global_variables_initializer())                     
                     
    
    

In [18]:
mse_history=[]
cost_history=[]
accuracy_history=[]

In [19]:
training_epochs=1000

In [20]:
for epoch in range(training_epochs):
    sess.run(train_step,feed_dict={x:x_train,y_:y_train})
    cost=sess.run(cross_entropy,feed_dict={x:x_train,y_:y_train})
    cost_history.append(np.append(cost_history,cost))
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    pred_y=sess.run(y,feed_dict={x:x_test})
    mse=tf.reduce_mean(tf.square(pred_y-y_test))
    mse_=sess.run(mse)
    mse_history.append(mse_)
    accuracy=sess.run(accuracy,feed_dict={x:x_train,y_:y_train})
    accuracy_history.append(accuracy)
    
    print('epoch:',epoch,'-','cost:',cost,'mse: ',mse_,'-','-train accuracy:',accuracy)

epoch: 0 - cost: 14.423 mse:  870.948671723 - -train accuracy: 0.492148
epoch: 1 - cost: 9.81883 mse:  648.277495597 - -train accuracy: 0.510514
epoch: 2 - cost: 13.0674 mse:  824.742540291 - -train accuracy: 0.492148
epoch: 3 - cost: 9.51903 mse:  639.498736882 - -train accuracy: 0.510248
epoch: 4 - cost: 11.7187 mse:  783.136033461 - -train accuracy: 0.492148
epoch: 5 - cost: 9.50597 mse:  643.436584133 - -train accuracy: 0.509449
epoch: 6 - cost: 10.35 mse:  744.171025237 - -train accuracy: 0.492148
epoch: 7 - cost: 9.75387 mse:  653.91212373 - -train accuracy: 0.509183
epoch: 8 - cost: 9.00779 mse:  709.703248609 - -train accuracy: 0.492148
epoch: 9 - cost: 10.167 mse:  667.10179829 - -train accuracy: 0.508917
epoch: 10 - cost: 7.72671 mse:  683.174758693 - -train accuracy: 0.492148
epoch: 11 - cost: 10.6809 mse:  680.102553605 - -train accuracy: 0.508651
epoch: 12 - cost: 6.48907 mse:  659.400348456 - -train accuracy: 0.492148
epoch: 13 - cost: 11.2283 mse:  692.29222034 - -train 

epoch: 112 - cost: 3.09396 mse:  511.698962078 - -train accuracy: 0.511578
epoch: 113 - cost: 4.66259 mse:  515.031147598 - -train accuracy: 0.525685
epoch: 114 - cost: 3.07316 mse:  509.240626045 - -train accuracy: 0.512377
epoch: 115 - cost: 4.60447 mse:  512.125779443 - -train accuracy: 0.527016
epoch: 116 - cost: 3.05276 mse:  506.816665729 - -train accuracy: 0.513442
epoch: 117 - cost: 4.54742 mse:  509.26107043 - -train accuracy: 0.528081
epoch: 118 - cost: 3.03221 mse:  504.429628646 - -train accuracy: 0.514506
epoch: 119 - cost: 4.49278 mse:  506.408469394 - -train accuracy: 0.528347
epoch: 120 - cost: 3.0119 mse:  502.018631216 - -train accuracy: 0.516369
epoch: 121 - cost: 4.43906 mse:  503.572505205 - -train accuracy: 0.529146
epoch: 122 - cost: 2.99193 mse:  499.693408135 - -train accuracy: 0.516636
epoch: 123 - cost: 4.38865 mse:  500.902868713 - -train accuracy: 0.529678
epoch: 124 - cost: 2.9714 mse:  497.47223281 - -train accuracy: 0.516369
epoch: 125 - cost: 4.33839 ms

epoch: 223 - cost: 2.77294 mse:  394.704090693 - -train accuracy: 0.56082
epoch: 224 - cost: 2.12603 mse:  407.483507145 - -train accuracy: 0.541656
epoch: 225 - cost: 2.75161 mse:  392.931639201 - -train accuracy: 0.561884
epoch: 226 - cost: 2.11236 mse:  405.832090543 - -train accuracy: 0.542188
epoch: 227 - cost: 2.72892 mse:  391.11446505 - -train accuracy: 0.562683
epoch: 228 - cost: 2.09826 mse:  404.158614116 - -train accuracy: 0.542986
epoch: 229 - cost: 2.70712 mse:  389.296976623 - -train accuracy: 0.563215
epoch: 230 - cost: 2.08467 mse:  402.487639145 - -train accuracy: 0.542986
epoch: 231 - cost: 2.68591 mse:  387.59331088 - -train accuracy: 0.564546
epoch: 232 - cost: 2.07149 mse:  400.932792016 - -train accuracy: 0.543253
epoch: 233 - cost: 2.66437 mse:  385.892005746 - -train accuracy: 0.565345
epoch: 234 - cost: 2.05782 mse:  399.353121055 - -train accuracy: 0.543253
epoch: 235 - cost: 2.64412 mse:  384.281362618 - -train accuracy: 0.566409
epoch: 236 - cost: 2.04575 m

epoch: 333 - cost: 1.90094 mse:  320.013844103 - -train accuracy: 0.593026
epoch: 334 - cost: 1.55848 mse:  335.774889091 - -train accuracy: 0.566942
epoch: 335 - cost: 1.8904 mse:  319.078348716 - -train accuracy: 0.593293
epoch: 336 - cost: 1.5508 mse:  334.844211459 - -train accuracy: 0.567208
epoch: 337 - cost: 1.8797 mse:  318.169478417 - -train accuracy: 0.593559
epoch: 338 - cost: 1.54315 mse:  333.930960534 - -train accuracy: 0.567208
epoch: 339 - cost: 1.86808 mse:  317.243095694 - -train accuracy: 0.593293
epoch: 340 - cost: 1.53535 mse:  332.996322781 - -train accuracy: 0.567474
epoch: 341 - cost: 1.85735 mse:  316.365093206 - -train accuracy: 0.593559
epoch: 342 - cost: 1.5281 mse:  332.128395343 - -train accuracy: 0.568006
epoch: 343 - cost: 1.84673 mse:  315.459573813 - -train accuracy: 0.593559
epoch: 344 - cost: 1.52024 mse:  331.251371095 - -train accuracy: 0.568273
epoch: 345 - cost: 1.83611 mse:  314.568777619 - -train accuracy: 0.594357
epoch: 346 - cost: 1.51269 ms

epoch: 443 - cost: 1.41996 mse:  276.354225104 - -train accuracy: 0.615651
epoch: 444 - cost: 1.21726 mse:  290.699914628 - -train accuracy: 0.583977
epoch: 445 - cost: 1.41347 mse:  275.693647864 - -train accuracy: 0.615917
epoch: 446 - cost: 1.21228 mse:  289.995111817 - -train accuracy: 0.584775
epoch: 447 - cost: 1.40675 mse:  275.029183124 - -train accuracy: 0.616715
epoch: 448 - cost: 1.20728 mse:  289.279976872 - -train accuracy: 0.585041
epoch: 449 - cost: 1.40031 mse:  274.363160446 - -train accuracy: 0.616982
epoch: 450 - cost: 1.20272 mse:  288.557297132 - -train accuracy: 0.585574
epoch: 451 - cost: 1.39397 mse:  273.694656637 - -train accuracy: 0.617248
epoch: 452 - cost: 1.19813 mse:  287.830213651 - -train accuracy: 0.586372
epoch: 453 - cost: 1.38806 mse:  273.025795642 - -train accuracy: 0.617248
epoch: 454 - cost: 1.1936 mse:  287.120423508 - -train accuracy: 0.58584
epoch: 455 - cost: 1.38167 mse:  272.369529636 - -train accuracy: 0.618312
epoch: 456 - cost: 1.18894 

epoch: 553 - cost: 1.13105 mse:  245.812260255 - -train accuracy: 0.635614
epoch: 554 - cost: 1.00196 mse:  257.977955458 - -train accuracy: 0.597019
epoch: 555 - cost: 1.12687 mse:  245.37745368 - -train accuracy: 0.635614
epoch: 556 - cost: 0.998901 mse:  257.509698348 - -train accuracy: 0.597285
epoch: 557 - cost: 1.12299 mse:  244.957110119 - -train accuracy: 0.636146
epoch: 558 - cost: 0.995792 mse:  257.069593091 - -train accuracy: 0.597019
epoch: 559 - cost: 1.11913 mse:  244.566596937 - -train accuracy: 0.636678
epoch: 560 - cost: 0.99289 mse:  256.659132705 - -train accuracy: 0.597019
epoch: 561 - cost: 1.11552 mse:  244.171240789 - -train accuracy: 0.636944
epoch: 562 - cost: 0.990202 mse:  256.244870361 - -train accuracy: 0.597285
epoch: 563 - cost: 1.11171 mse:  243.768296523 - -train accuracy: 0.637211
epoch: 564 - cost: 0.987184 mse:  255.818831062 - -train accuracy: 0.597551
epoch: 565 - cost: 1.10739 mse:  243.368977106 - -train accuracy: 0.637477
epoch: 566 - cost: 0.9

epoch: 662 - cost: 0.861398 mse:  238.88049043 - -train accuracy: 0.616449
epoch: 663 - cost: 0.942289 mse:  228.073355026 - -train accuracy: 0.653447
epoch: 664 - cost: 0.858962 mse:  238.597820943 - -train accuracy: 0.617248
epoch: 665 - cost: 0.939003 mse:  227.831424102 - -train accuracy: 0.653713
epoch: 666 - cost: 0.856562 mse:  238.319514527 - -train accuracy: 0.61778
epoch: 667 - cost: 0.935932 mse:  227.58529361 - -train accuracy: 0.654512
epoch: 668 - cost: 0.854126 mse:  238.036523192 - -train accuracy: 0.618046
epoch: 669 - cost: 0.932695 mse:  227.342039177 - -train accuracy: 0.655044
epoch: 670 - cost: 0.851706 mse:  237.751644058 - -train accuracy: 0.618312
epoch: 671 - cost: 0.929486 mse:  227.09717664 - -train accuracy: 0.655044
epoch: 672 - cost: 0.849471 mse:  237.476634639 - -train accuracy: 0.618579
epoch: 673 - cost: 0.926537 mse:  226.859045265 - -train accuracy: 0.65531
epoch: 674 - cost: 0.847415 mse:  237.219354979 - -train accuracy: 0.619111
epoch: 675 - cost

epoch: 771 - cost: 0.806098 mse:  217.007792683 - -train accuracy: 0.675273
epoch: 772 - cost: 0.75469 mse:  225.685285639 - -train accuracy: 0.632952
epoch: 773 - cost: 0.804075 mse:  216.826594229 - -train accuracy: 0.675539
epoch: 774 - cost: 0.753064 mse:  225.475126926 - -train accuracy: 0.634017
epoch: 775 - cost: 0.802019 mse:  216.636335314 - -train accuracy: 0.675539
epoch: 776 - cost: 0.751473 mse:  225.265191751 - -train accuracy: 0.634815
epoch: 777 - cost: 0.800061 mse:  216.460394044 - -train accuracy: 0.675805
epoch: 778 - cost: 0.749924 mse:  225.057605581 - -train accuracy: 0.635081
epoch: 779 - cost: 0.79808 mse:  216.292951387 - -train accuracy: 0.676337
epoch: 780 - cost: 0.748405 mse:  224.858627059 - -train accuracy: 0.636944
epoch: 781 - cost: 0.796198 mse:  216.12975482 - -train accuracy: 0.676604
epoch: 782 - cost: 0.747028 mse:  224.665116502 - -train accuracy: 0.638009
epoch: 783 - cost: 0.794413 mse:  215.965377298 - -train accuracy: 0.67687
epoch: 784 - cos

epoch: 880 - cost: 0.681609 mse:  215.730363813 - -train accuracy: 0.659835
epoch: 881 - cost: 0.713427 mse:  208.37604907 - -train accuracy: 0.692308
epoch: 882 - cost: 0.680465 mse:  215.553894871 - -train accuracy: 0.660634
epoch: 883 - cost: 0.711949 mse:  208.23386201 - -train accuracy: 0.692574
epoch: 884 - cost: 0.679223 mse:  215.389327223 - -train accuracy: 0.661166
epoch: 885 - cost: 0.710416 mse:  208.103812565 - -train accuracy: 0.692574
epoch: 886 - cost: 0.677976 mse:  215.231268242 - -train accuracy: 0.661698
epoch: 887 - cost: 0.708946 mse:  207.967763806 - -train accuracy: 0.693639
epoch: 888 - cost: 0.676768 mse:  215.071954324 - -train accuracy: 0.66223
epoch: 889 - cost: 0.707462 mse:  207.824406832 - -train accuracy: 0.694171
epoch: 890 - cost: 0.675537 mse:  214.901402425 - -train accuracy: 0.662497
epoch: 891 - cost: 0.705922 mse:  207.691917904 - -train accuracy: 0.694703
epoch: 892 - cost: 0.67433 mse:  214.744482854 - -train accuracy: 0.663029
epoch: 893 - cos

epoch: 989 - cost: 0.643199 mse:  202.558026969 - -train accuracy: 0.709875
epoch: 990 - cost: 0.622671 mse:  208.370892704 - -train accuracy: 0.680596
epoch: 991 - cost: 0.642225 mse:  202.467509265 - -train accuracy: 0.710141
epoch: 992 - cost: 0.621851 mse:  208.260602954 - -train accuracy: 0.680596
epoch: 993 - cost: 0.641217 mse:  202.390543743 - -train accuracy: 0.710673
epoch: 994 - cost: 0.621042 mse:  208.165528031 - -train accuracy: 0.681129
epoch: 995 - cost: 0.640305 mse:  202.303257186 - -train accuracy: 0.71094
epoch: 996 - cost: 0.620281 mse:  208.060956809 - -train accuracy: 0.681395
epoch: 997 - cost: 0.639377 mse:  202.208446404 - -train accuracy: 0.711738
epoch: 998 - cost: 0.619488 mse:  207.942526849 - -train accuracy: 0.681395
epoch: 999 - cost: 0.638469 mse:  202.116205559 - -train accuracy: 0.71227


In [21]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('test accuracy:',sess.run(accuracy,feed_dict={x:x_test,y_:y_test}))

test accuracy: 0.706645


In [22]:
pred_y=sess.run(y,feed_dict={x:x_test})
mse=tf.reduce_mean(tf.square(pred_y-y_test))
print('mse:%.4f'%sess.run(mse))

mse:202.1162
